In [ ]:
!nvidia-smi

Mon Feb  2 11:19:26 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             37W /   70W |   13740MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()


In [2]:
# !pip install -q -U transformers accelerate bitsandbytes
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "Qwen/Qwen2.5-7B-Instruct"

# 1. Tải Tokenizer và Model với cấu hình 4-bit để không bị tràn RAM T4
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
    quantization_config={"load_in_4bit": True}, # Nén để chạy vừa GPU 16GB
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Tạo pipeline chat
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

In [4]:
# 3. Nội dung hội thoại
messages = [
    {"role": "system", "content": "Bạn là một trợ lý AI thông minh và nhiệt tình."},
    {"role": "user", "content": "Giải thích về lỗ đen vũ trụ bằng tiếng Việt một cách dễ hiểu cho trẻ em."}
]

# 4. Thực thi và in kết quả
outputs = pipe(messages, max_new_tokens=512*4, do_sample=True, temperature=0.9)
print(outputs[0]["generated_text"][-1]["content"])

Both `max_new_tokens` (=2048) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Làm quen với lỗ đen vũ trụ là một trải nghiệm thú vị, đặc biệt là khi chúng ta làm đơn giản và hấp dẫn như có thể:

Lỗ đen là một khu vực rất đặc biệt trong vũ trụ. Hãy tưởng tượng bạn đang ở trung tâm của một cái vòng xoáy khổng lồ - một lỗ xoáy lớn đến nỗi không gì có thể thoát ra khỏi nó. Lỗ đen là một điểm trong không gian nơi mà lực hấp dẫn trở nên mạnh mẽ đến mức không có gì - kể cả ánh sáng - có thể thoát ra khỏi nó.

Đôi khi, các ngôi sao lớn hoặc hành tinh lớn bị kéo vào lỗ đen. Điều này giống như khi bạn thả một quả bóng vào nước - quả bóng sẽ chìm xuống đáy và không bao giờ trở lại được.

Lỗ đen không thực sự "ăn" những thứ xung quanh chúng. Thay vào đó, chúng hấp thụ mọi thứ mà chúng có thể hấp thụ - bất cứ điều gì di chuyển quá gần. Chúng giống như một "khe hở" trong vũ trụ, hút mọi thứ vào đó!

Nhưng đừng lo lắng, các lỗ đen không ở gần Trái Đất. Chúng nằm ở rất xa và không gây nguy hiểm cho chúng ta.


In [3]:
# !pip install -q -U transformers accelerate

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "Qwen/Qwen2.5-7B-Instruct"

# 1. Load Model without quantization config
# We use torch.float16 (Standard Half-Precision) instead of 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,  # Standard FP16. Use torch.bfloat16 if on Ampere (A100)
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Create chat pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Example usage
messages = [{"role": "user", "content": "Hello, how are you?"}]
output = pipe(messages, max_new_tokens=128)
print(output[0]['generated_text'])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'role': 'user', 'content': 'Hello, how are you?'}, {'role': 'assistant', 'content': "Hello! I'm an AI assistant created by Alibaba Cloud, so I don't have personal feelings or experiences. I'm here to help you with any questions or information you might need. How can I assist you today?"}]


In [ ]:
# !pip install -q -U transformers accelerate bitsandbytes 

import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from huggingface_hub import login

# --- FIX 1: Login explicitly to stop the HF_TOKEN warning ---
# Replace 'hf_...' with your actual token from https://huggingface.co/settings/tokens
# If you don't have one or don't want to add it, you can delete this line (Qwen is public).
# login(token="hf_YOUR_TOKEN_HERE") 

# Suppress minor warnings to keep output clean
warnings.filterwarnings("ignore")

model_name = "Qwen/Qwen2.5-7B-Instruct"

# --- FIX 2: Force Float16 to stop the MatMul cast warning ---
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,  # Force standard FP16 to match 8-bit requirements
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)



`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Passing `generation_config` together with generation-related arguments=({'max_new_tokens', 'temperature', 'do_sample'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=2048) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- Model Answer ---
Quantum physics is the branch of physics that deals with phenomena at microscopic scales, such as the behavior of subatomic particles, where the principles of classical mechanics break down and probabilities play a fundamental role in the description of physical properties.


In [4]:
# --- FIX 3: Clean Pipeline Call ---
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

messages = [{"role": "user", "content": "Explain quantum physics "}]

# We use return_full_text=False so you only get the answer, not the prompt repeated
output = pipe(
    messages, 
    max_new_tokens=512*10, 
    do_sample=True, 
    temperature=0.7,
    return_full_text=False
)

print("\n--- Model Answer ---")
print(output[0]['generated_text'])

Both `max_new_tokens` (=5120) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- Model Answer ---
Quantum physics, also known as quantum mechanics, is a fundamental theory in physics that describes the physical properties of nature at the scale of atoms and subatomic particles (such as electrons, protons, and photons). It provides a mathematical description of much of the dual particle-like and wave-like behavior and interactions of energy and matter.

### Key Concepts:

1. **Wave-Particle Duality**:
   - Quantum objects exhibit both particle-like and wave-like behavior. For example, light can be described as either photons (particles) or electromagnetic waves, depending on the experiment.
   
2. **Quantization**:
   - Energy is not continuous but comes in discrete packets called quanta. This concept was introduced by Max Planck to explain blackbody radiation.

3. **Superposition**:
   - Quantum systems can exist in multiple states simultaneously until they are measured. For instance, an electron in an atom can be in a superposition of different energy levels 